In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms


In [2]:
class CNN(nn.Module):
    def __init__(self, in_channels, num_classes):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(
            in_channels=in_channels,
            out_channels=8,
            kernel_size=(3, 3),
            stride=(1, 1),
            padding=(1, 1),
        )
        self.pool = nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2))
        self.conv2 = nn.Conv2d(
            in_channels=8,
            out_channels=16,
            kernel_size=(3, 3),
            stride=(1, 1),
            padding=(1, 1),
        )
        self.linear = nn.Linear(784, num_classes)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = x.reshape(x.shape[0], -1)
        return self.linear(x)

In [3]:
class Network(nn.Module):
    def __init__(self, input_size, num_classes):
        super(Network, self).__init__()
        self.L1 = nn.Linear(input_size, 50)
        self.L2 = nn.Linear(50, num_classes)

    def forward(self, x):
        x = F.relu(self.L1(x))
        return self.L2(x)


In [4]:
in_channels = 1
batch_size = 64
input_size = 784
num_classes = 10
learning_rate = 1e-3
num_epochs = 5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
train_dataset = datasets.MNIST(
    root="./dataset/", train=True, transform=transforms.ToTensor(), download=True
)
test_dataset = datasets.MNIST(
    root="./dataset/", train=False, transform=transforms.ToTensor(), download=True
)

In [6]:
train_loader = DataLoader(dataset = train_dataset, batch_size=batch_size, shuffle = True)
test_loader = DataLoader(dataset = test_dataset, batch_size=batch_size, shuffle=True)

In [7]:
modelCNN = CNN(in_channels=in_channels, num_classes=num_classes).to(device=device)
modelFullLinear = Network(input_size=input_size, num_classes=num_classes).to(device)

In [8]:
criterion = nn.CrossEntropyLoss()
optimizerCNN = optim.Adam(modelCNN.parameters(), lr=learning_rate)
optimizerLinear = optim.Adam(modelFullLinear.parameters(), lr=learning_rate)

In [9]:
for epoch in range(num_epochs):
    for idx, (data, targets) in enumerate(train_loader):
        data = data.to(device)
        targets = targets.to(device)
        scores = modelCNN(data)
        loss = criterion(scores, targets)

        optimizerCNN.zero_grad()
        loss.backward()

        optimizerCNN.step()
    print((100 * epoch) / num_epochs)

0.0
5.0
10.0
15.0
20.0
25.0
30.0
35.0
40.0
45.0
50.0
55.0
60.0
65.0
70.0
75.0
80.0
85.0
90.0
95.0


In [10]:
# for epoch in range(num_epochs):
#     for batch_idx, (data, targets) in enumerate(train_loader):
#         data = data.to(device)
#         targets = targets.to(device)
#         data = data.reshape(data.shape[0], -1)

#         scores = modelFullLinear(data)
#         loss = criterion(scores, targets)

#         optimizerLinear.zero_grad()
#         loss.backward()

#         optimizerLinear.step()

In [17]:
def get_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device)
            y = y.to(device)
            # x = x.reshape(x.shape[0], -1)
            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)
    print(f"Accuracy : {(num_correct / num_samples)*100:.2f}%")

In [18]:
get_accuracy(test_loader, modelCNN)

Accuracy : 98.77%


### Saving and Loading.

In [13]:
torch.save(modelCNN.state_dict(), "mnistModel.pt")

In [14]:
p = CNN(num_classes=num_classes, in_channels=in_channels)

In [15]:
p.load_state_dict(torch.load("./mnistModel.pt"))

<All keys matched successfully>

In [16]:
p.eval()

CNN(
  (conv1): Conv2d(1, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (linear): Linear(in_features=784, out_features=10, bias=True)
)